In [1]:
'''
Terminology:
	Episode: one go of the car trying to "live" in the simulation and earn max rewards.
				Episode start from spawning a new car and ends with either car crashing or episode duration limit running out

	Timestep: one frame through simulation: the car gets a obs, reward from prior step and then it makes a decision on control input and sends it to simulation

	Reward logic: each timestep a logic is applied to calculate a reward from latest step. 
				This logic represents you describing the desired behaviour the car needs to learn.

	Policy/model: our objective, what we need learn as part of RL. This is the latest set of rules on what to do at a given camera image.

	Iterations: RL training sessions (multiple episodes and timesteps) when a policy/model is saved. So the policy is changed throughout one iteration
				but then saved in a new file at the end of iteration. This allows to test all models later at different stages of training  
'''	

from stable_baselines3 import PPO #PPO
from stable_baselines3.common.env_checker import check_env

import os
from environment import CarEnv
import time

In [2]:

print('This is the start of training script')

print('setting folders for logs and models')


print('connecting to env..')


env = CarEnv()  # Try check_env(env, warn=True) 
check_env(env, warn=True) 
env.reset()


This is the start of training script
setting folders for logs and models
connecting to env..


2024-10-03 16:12:07,447 - INFO - Spawning vehicle at: Location(x=174.330048, y=237.429993, z=0.500000)
2024-10-03 16:12:07,452 - INFO - Current waypoint: Location(x=173.521225, y=236.948929, z=0.000000)
2024-10-03 16:12:07,453 - INFO - Target waypoint: Location(x=170.521225, y=236.947098, z=0.000000)
2024-10-03 16:12:07,455 - INFO - lateral_distance: -0.16019377572214058, speed: 0.0, heading: 0.00019132826063368057
2024-10-03 16:12:07,462 - INFO - Spawning vehicle at: Location(x=189.929993, y=142.190002, z=0.500000)
2024-10-03 16:12:07,465 - INFO - Current waypoint: Location(x=189.707352, y=142.087311, z=0.000000)
2024-10-03 16:12:07,466 - INFO - Target waypoint: Location(x=189.703278, y=145.087296, z=0.000000)
2024-10-03 16:12:07,468 - INFO - lateral_distance: -0.07425969993610584, speed: 0.0, heading: 0.0004326714409722222
2024-10-03 16:12:07,498 - INFO - Spawning vehicle at: Location(x=88.710045, y=237.429993, z=0.500000)
2024-10-03 16:12:07,502 - INFO - Current waypoint: Location(x

(array([0.0361    , 0.        , 0.00032294], dtype=float32), {})

In [3]:

print('Env has been reset as part of launch')

model = PPO(
    'MlpPolicy', 
    env, 
    verbose=1, 
    learning_rate=0.0003, 
    n_steps=1024, 
    batch_size=64, 
    clip_range=0.1, 
    gamma=0.99, 
    normalize_advantage=True,
)

	

Env has been reset as part of launch
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [3]:


# Directory containing saved models
models_dir = "models/0210/1727878874"
logdir = "logs/0210/1727878874"

# Ensure the directories exist
assert os.path.exists(models_dir), f"Model directory {models_dir} does not exist!"
assert os.path.exists(logdir), f"Log directory {logdir} does not exist!"

# List all the saved models in the directory
model_files = [f for f in sorted(os.listdir(models_dir)) if f.endswith('.zip')]
assert len(model_files) > 0, "No models found in the models directory!"

# Load the environment
env = CarEnv()
check_env(env)
env.reset()


2024-10-03 16:12:14,440 - INFO - Spawning vehicle at: Location(x=173.870056, y=105.550011, z=0.500000)
2024-10-03 16:12:14,447 - INFO - Current waypoint: Location(x=172.498642, y=105.426109, z=0.000000)
2024-10-03 16:12:14,448 - INFO - Target waypoint: Location(x=169.498642, y=105.428146, z=0.000000)
2024-10-03 16:12:14,451 - INFO - lateral_distance: -0.041611417809444225, speed: 0.0, heading: -0.00021828545464409722
2024-10-03 16:12:14,460 - INFO - Spawning vehicle at: Location(x=135.879990, y=215.269989, z=0.500000)
2024-10-03 16:12:14,468 - INFO - Current waypoint: Location(x=136.134064, y=215.182022, z=0.000000)
2024-10-03 16:12:14,469 - INFO - Target waypoint: Location(x=136.137115, y=212.182037, z=0.000000)
2024-10-03 16:12:14,472 - INFO - lateral_distance: -0.08466514813394915, speed: 0.0, heading: 0.0003216637505425347
2024-10-03 16:12:14,490 - INFO - Spawning vehicle at: Location(x=165.090042, y=187.119949, z=0.500000)
2024-10-03 16:12:14,497 - INFO - Current waypoint: Locatio

(array([0.13113587, 0.        , 0.00019438], dtype=float32), {})

In [4]:

# Select the most recent model to test
latest_model_path = os.path.join(models_dir, model_files[-1])
print(f"Loading model from {latest_model_path}")
model = PPO.load(latest_model_path, env=env)


Loading model from models/0210/1727878874\750000.zip
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:

# Number of episodes to test
num_test_episodes = 1

for episode in range(num_test_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    timestep = 0

    print(f"Starting Episode {episode + 1}")

    while not done:
        # Get the action from the model
        action, _states = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        print(f"Action: {action}")
        print(f"Obs: {obs}")
        print(f"Reward: {reward}")
        total_reward += reward
        timestep += 1
        if done or truncated:
            break

    print(f"Episode {episode + 1} finished. Total Reward: {total_reward} after {timestep} timesteps.\n")

print("Testing completed.")
env.close()


2024-10-03 16:12:17,528 - INFO - Spawning vehicle at: Location(x=136.110016, y=306.420044, z=0.500000)
2024-10-03 16:12:17,535 - INFO - Current waypoint: Location(x=136.659302, y=306.559845, z=0.000000)
2024-10-03 16:12:17,536 - INFO - Target waypoint: Location(x=139.662720, y=306.556549, z=0.000000)
2024-10-03 16:12:17,540 - INFO - lateral_distance: -0.04712866668060092, speed: 0.0, heading: -0.000436303640405337


Starting Episode 1
Action: [8 2]
Obs: [-0.04712867  0.         -0.0004363 ]
Reward: -5.0
Action: [5 1]
Obs: [-0.04712867  0.01666667 -0.0004363 ]
Reward: -5.0
Action: [8 3]
Obs: [-0.04712867  0.01666667 -0.0004363 ]
Reward: -5.0
Action: [1 3]
Obs: [-0.04712867  0.01666667 -0.0004363 ]
Reward: -5.0
Action: [1 2]
Obs: [-0.04712867  0.03333334 -0.0004363 ]
Reward: -5.0
Action: [5 1]
Obs: [-0.04712867  0.03333334 -0.0004363 ]
Reward: -5.0
Action: [6 1]
Obs: [-0.04712867  0.03333334 -0.0004363 ]
Reward: -5.0
Action: [8 1]
Obs: [-0.04712867  0.05       -0.0004363 ]
Reward: -5.0
Action: [1 1]
Obs: [-0.04712867  0.05       -0.0004363 ]
Reward: -5.0
Action: [8 2]
Obs: [-0.04712867  0.05       -0.0004363 ]
Reward: -5.0
Action: [8 2]
Obs: [-0.04712867  0.06666667 -0.0004363 ]
Reward: -5.0
Action: [3 2]
Obs: [-0.04712867  0.06666667 -0.0004363 ]
Reward: -5.0
Action: [2 0]
Obs: [-0.04712867  0.06666667 -0.0004363 ]
Reward: -5.0
Action: [2 2]
Obs: [-0.04712867  0.08333334 -0.0004363 ]
Reward: -5.0
A

2024-10-03 16:12:19,450 - INFO - lateral_distance: -0.0470385668497924, speed: 0.05, heading: -0.0007792796111769147


Action: [7 1]
Obs: [-0.04705936  0.01666667 -0.00048587]
Reward: -5.0
Action: [2 2]
Obs: [-0.04708747  0.01666667 -0.00051017]
Reward: -5.0
Action: [0 3]
Obs: [-0.04713534  0.03333334 -0.00053177]
Reward: -5.0
Action: [6 0]
Obs: [-0.04717257  0.03333334 -0.00054763]
Reward: -5.0
Action: [3 1]
Obs: [-0.04719935  0.03333334 -0.00053129]
Reward: -5.0
Action: [8 1]
Obs: [-0.0471952   0.05       -0.00053812]
Reward: -5.0
Action: [6 3]
Obs: [-0.04718055  0.05       -0.00062026]
Reward: -5.0
Action: [8 2]
Obs: [-0.04709433  0.05       -0.00070369]
Reward: -5.0
s: 3.0, loc: Location(x=136.195694, y=306.420197, z=0.220957)
dest: Location(x=139.662720, y=306.556549, z=0.000000)
vehicle_transform: Transform(Location(x=136.195694, y=306.420197, z=0.220957), Rotation(pitch=0.042907, yaw=0.061553, roll=-0.002655)), self.dest_waypoint: Waypoint(Transform(Location(x=139.662720, y=306.556549, z=0.000000), Rotation(pitch=0.000000, yaw=-0.078718, roll=0.000000))), velocity: Vector3D(x=1.067896, y=0.02370

2024-10-03 16:12:20,996 - INFO - lateral_distance: -0.023859102530424014, speed: 0.2, heading: -0.008711344955696


...

Action: [3 2]
Obs: [ 0.01559309  0.36666667 -0.00528958]
Reward: 10.0
Action: [1 3]
Obs: [ 0.01609981  0.36666667 -0.00564092]
Reward: 10.0
Action: [8 2]
Obs: [ 0.01668818  0.36666667 -0.00576251]
Reward: 10.0
Action: [2 1]
Obs: [ 0.01714398  0.36666667 -0.00605852]
Reward: 10.0
Action: [8 2]
Obs: [ 0.0177018   0.36666667 -0.00613754]
Reward: 10.0
s: 22.0, loc: Location(x=191.583160, y=306.539978, z=0.301692)
dest: Location(x=163.662704, y=306.523590, z=0.000000)
vehicle_transform: Transform(Location(x=191.583160, y=306.539978, z=0.301692), Rotation(pitch=0.913203, yaw=1.089075, roll=-1.787292)), self.dest_waypoint: Waypoint(Transform(Location(x=163.662704, y=306.523590, z=0.000000), Rotation(pitch=0.000000, yaw=-0.078718, roll=0.000000))), velocity: Vector3D(x=6.228670, y=0.196798, z=0.005401)
obs: [ 0.01824913  0.36666667 -0.00648774]
Action: [5 2]
Obs: [ 0.01824913  0.36666667 -0.00648774]
Reward: 10.0
Action: [6 2]
Obs: [ 0.01893915  0.36666667 -0.00666958]
Reward: 10.0
Action: [3 

2024-10-03 16:12:41,718 - INFO - lateral_distance: -0.06036763039815088, speed: 0.3333333333333333, heading: 0.009542094709144699


Action: [0 2]
Obs: [-0.05925506  0.35        0.01585313]
Reward: 10.0
Action: [7 2]
Obs: [-0.05960461  0.35        0.01497524]
Reward: 10.0
Action: [7 0]
Obs: [-0.05973037  0.35        0.01442237]
Reward: 10.0
Action: [3 0]
Obs: [-0.05993756  0.35        0.01349751]
Reward: 10.0
Action: [7 0]
Obs: [-0.0599515   0.35        0.01294413]
Reward: 10.0
Action: [0 1]
Obs: [-0.06010789  0.35        0.01204098]
Reward: 10.0
Action: [7 0]
Obs: [-0.06009139  0.33333334  0.01153371]
Reward: 10.0
Action: [0 2]
Obs: [-0.06022753  0.33333334  0.01069787]
Reward: 10.0
Action: [8 2]
Obs: [-0.06020091  0.33333334  0.01027537]
Reward: 10.0
s: 20.0, loc: Location(x=197.639450, y=306.295807, z=0.322973)
dest: Location(x=163.662704, y=306.523590, z=0.000000)
vehicle_transform: Transform(Location(x=197.639450, y=306.295807, z=0.322973), Rotation(pitch=-0.119419, yaw=-1.796295, roll=-0.042358)), self.dest_waypoint: Waypoint(Transform(Location(x=163.662704, y=306.523590, z=0.000000), Rotation(pitch=0.000000, 

2024-10-03 16:12:42,751 - INFO - Collision detected, ending episode


Action: [8 0]
Obs: [-0.0766603   0.31666666  0.00884969]
Reward: 4.999998222222223
Action: [4 2]
Obs: [-0.07719499  0.31666666  0.00882917]
Reward: 4.999998222222223
Action: [1 3]
Obs: [-0.07766913  0.31666666  0.00904737]
Reward: 4.999998222222223
Action: [5 2]
Obs: [-0.07820398  0.31666666  0.00898651]
Reward: 4.999998222222223
Action: [0 2]
Obs: [-0.07868849  0.3         0.0091786 ]
Reward: 4.999991
Action: [0 0]
Obs: [-0.07931499  0.31666666  0.00923539]
Reward: 4.999998222222223
Action: [6 2]
Obs: [-0.07985043  0.3         0.00948445]
Reward: 4.999991
Action: [3 1]
Obs: [-0.07985043  0.3         0.00948445]
Reward: -1005.0
Episode 1 finished. Total Reward: 7039.942743000017 after 1345 timesteps.

Testing completed.


## Train

In [ ]:


TIMESTEPS = 250000 # how long is each training iteration - individual steps
iters = 0
while iters<4:  # how many training iterations you want 
	iters += 1
	print('Iteration ', iters,' is to commence...')
	model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO" )
	print('Iteration ', iters,' has been trained')
	model.save(f"{models_dir}/{TIMESTEPS*iters}")
